In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.abspath('..'))

In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.datasets import make_circles

# Tensor class
from pynet.tensor import Tensor

# Neural network modules
from pynet.nn.abstract import Module
from pynet.nn.sequential import Sequential
from pynet.nn.linear import Linear
from pynet.nn.relu import ReLU
from pynet.nn.sigmoid import Sigmoid

# Datasets
from pynet.data.in_memory import InMemoryDataset

# Loss functions
from pynet.loss.bce import BinaryCrossEntropy

# Optimizers
from pynet.optimizers.sgd import SGD

# Weight initializers
from pynet.initializers.he_normal import HeNormal

# Trainer and training/testing callbacks
from pynet.training.trainer import Trainer
from pynet.training.callbacks.abstract import Callback
from pynet.training.callbacks.print import PrintCallback
from pynet.training.history import History

In [4]:
class GifCallback(Callback):
    def __init__(self, model: Module, x: np.ndarray, y: np.ndarray, imgdir: str) -> None:
        super().__init__()

        self.__model = model
        self.__x = x
        self.__y = y
        self.__imgdir = imgdir

    def on_train_begin(self) -> None:
        self.__create_boundary(-1)
    
    def on_epoch_end(self, history: History) -> None:
        epoch = history.rows[-1].epoch
        self.__create_boundary(epoch)
    
    def __create_boundary(self, epoch: int) -> None:
        model = self.__model
        x = self.__x
        y = self.__y

        x_min, x_max = x[:, 0, 0].min() - 0.1, x[:, 0, 0].max() + 0.1
        y_min, y_max = x[:, 1, 0].min() - 0.1, x[:, 1, 0].max() + 0.1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

        grid = np.c_[xx.ravel(), yy.ravel()]
        grid = np.expand_dims(grid, axis=2)

        preds = [model.forward(Tensor(grid[i])) for i in range(len(grid))]
        z = np.array([round(pred.ndarray.item()) for pred in preds])
        z = z.reshape(xx.shape)

        fig, ax = plt.subplots(figsize=(10, 10))
        c_dict = {0: "tab:green", 1: "tab:red"}
        colors =list(np.vectorize(c_dict.get)(y))

        ax.contourf(xx, yy, z, 1, alpha=0.4, colors=["tab:green", "tab:red"])
        ax.scatter(x[:,0,0], x[:,1,0], c=colors)
        ax.set_title(f"Epoch: {epoch + 1}")

        os.makedirs(self.__imgdir, exist_ok=True) 

        plt.savefig(os.path.join(self.__imgdir, f"epoch_{epoch}.jpg"), format="jpg")
        plt.close(fig)

In [6]:
X, y = make_circles(n_samples=1000, noise=0.025)
# inputs to neural net must be of shape [n, 1]
X = np.expand_dims(X, axis=2)
epochs = 20
imgdir = ".\\training_visualization\\classification"

model = Sequential([
    Linear(inputs=2, neurons=16, initializer=HeNormal()),
    ReLU(),
    Linear(inputs=16, neurons=1, initializer=HeNormal()),
    Sigmoid()
])

train_dataset = InMemoryDataset(X, y)
loss_f = BinaryCrossEntropy()
sgd = SGD(learning_rate=0.01, momentum=0.9)
callbacks = [PrintCallback(), GifCallback(model, X, y, imgdir)]
trainer = Trainer()

In [7]:
trainer.train(
    model=model,
    train_dataset=train_dataset,
    val_dataset=None,
    loss_f=loss_f,
    optimizer=sgd,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 0001 -> train_loss: 0.6905, train_accuracy: 0.5420
Epoch 0002 -> train_loss: 0.6600, train_accuracy: 0.6440
Epoch 0003 -> train_loss: 0.6414, train_accuracy: 0.7470
Epoch 0004 -> train_loss: 0.6162, train_accuracy: 0.7470
Epoch 0005 -> train_loss: 0.5970, train_accuracy: 0.8170
Epoch 0006 -> train_loss: 0.5688, train_accuracy: 0.8400
Epoch 0007 -> train_loss: 0.5349, train_accuracy: 0.8890
Epoch 0008 -> train_loss: 0.4995, train_accuracy: 0.9240
Epoch 0009 -> train_loss: 0.4612, train_accuracy: 0.9530
Epoch 0010 -> train_loss: 0.4280, train_accuracy: 0.9710
Epoch 0011 -> train_loss: 0.3895, train_accuracy: 0.9790
Epoch 0012 -> train_loss: 0.3559, train_accuracy: 0.9910
Epoch 0013 -> train_loss: 0.3238, train_accuracy: 0.9890
Epoch 0014 -> train_loss: 0.2931, train_accuracy: 0.9940
Epoch 0015 -> train_loss: 0.2668, train_accuracy: 0.9940
Epoch 0016 -> train_loss: 0.2396, train_accuracy: 0.9960
Epoch 0017 -> train_loss: 0.2161, train_accuracy: 0.9980
Epoch 0018 -> train_loss: 0.198

In [11]:
img, *imgs = [Image.open(os.path.join(imgdir, f"epoch_{i}.jpg")) for i in range(-1, epochs)]
img.save(os.path.join(imgdir, "training.gif"), format="GIF", append_images=imgs, save_all=True, duration=500, loop=0)